### Reading Satellite Data with Rasterio

In [33]:
import os
import rasterio

In [2]:
image_file = 'data/20190321_174348_0f1a_3B_AnalyticMS.tif'
satdat = rasterio.open(image_file)
print(satdat)

<open DatasetReader name='data/20190321_174348_0f1a_3B_AnalyticMS.tif' mode='r'>


In [5]:
print(f'raster name: {satdat.name}')
print(f'number of bands: {satdat.count}')
print(f'band indexes: {satdat.indexes}') # indexes in rasterio are 1-based

raster name: data/20190321_174348_0f1a_3B_AnalyticMS.tif
number of bands: 4
band indexes: (1, 2, 3, 4)


In [6]:
# bands in rasterio are organized as b, g, r instead of r, g, b
blue, green, red, nir = satdat.read()
print(blue.dtype)

uint16


In [7]:
width, height = blue.shape
print(f'width(px): {width}\nheight(px):{height}')

width(px): 4213
height(px):8341


### Extract metadata information from a satellite image

In [8]:
img = rasterio.open('data/20160831_180302_0e26_3B_Visual.tif')

In [9]:
print(img.bounds)

BoundingBox(left=623577.0, bottom=4199985.0, right=651732.0, top=4214037.0)


In [12]:
# Get the image dimensions in map units (e.g. meters)
bounds = img.bounds
width_in_projected_units = bounds.right - bounds.left
height_in_projected_units = bounds.top - bounds.bottom
print(f'width: {width_in_projected_units}, height: {height_in_projected_units}')
print(f'rows: {img.height}, columns: {img.width}')

width: 28155.0, height: 14052.0
rows: 4684, columns: 9385


In [13]:
pixel_size_x = width_in_projected_units / img.width
pixel_size_y = height_in_projected_units / img.height
print(f'pixel resolution\nX: {pixel_size_x}, Y: {pixel_size_y}')

pixel resolution
X: 3.0, Y: 3.0


In [15]:
# Coordinate Reference System
img.crs

CRS.from_epsg(32610)

In [18]:
img.transform

Affine(3.0, 0.0, 623577.0,
       0.0, -3.0, 4214037.0)

##### Convering pixel coordinates to world coordinates

In [17]:
# upper left pixel
row_min = 0
col_min = 0

# bottom right pixel
row_max = img.height - 1
col_max = img.width - 1

# transform the coordinates using the dataset's affine transform
top_left = img.transform * (row_min, col_min)
bottom_right = img.transform * (row_max, col_max)

print(f'top left coordinate: {top_left}')
print(f'bottom right coordinate: {bottom_right}')

top left coordinate: (623577.0, 4214037.0)
bottom right coordinate: (637626.0, 4185885.0)


In [27]:

# All the metadata necessary to create a raster of the same properties
img.profile

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 9385, 'height': 4684, 'count': 4, 'crs': CRS.from_epsg(32610), 'transform': Affine(3.0, 0.0, 623577.0,
       0.0, -3.0, 4214037.0), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}

In [29]:
# img.profile.data returns a dictionary of the metadata
img.profile.data.get('driver')

'GTiff'

### Copying a raster dataset

In [32]:
# read all bands from the source image as a numpy array
data = img.read()
profile = img.profile
# update profile information (JPEG will compress the data)
profile['compress'] = 'JPEG'
# write the data into a new raster file
with rasterio.open('output/compressed_example.tif', 'w', **profile) as ds:
    ds.write(data)

In [35]:
from humanize import naturalsize as sz

In [37]:
## get the size of the new raster
orig_size = os.path.getsize(img.name)
new_size = os.path.getsize('output/compressed_example.tif')
print(f'original image: {sz(orig_size)}\nnew image: {sz(new_size)}')

original image: 69.1 MB
new image: 13.6 MB
